# AgentKit SDK Tool Tutorial 01: 调用沙箱工具使用教程

本教程可以**单独使用**，目标是在一个 Notebook 中完整跑通：

1. 安装与环境准备
2. 使用 `sdk.init_project` 初始化一个示例 Agent 项目 
3. 使用 `sdk.tools.client.create_tool` 创建一个 `"All-in-one"` 沙箱工具
4. 使用 `sdk.tools.client.add_tool` 为 Agent 项目集成沙箱工具
5. 调用 Agent 项目，在沙箱环境中完成代码执行任务


### 安装 AgentKit SDK

```bash
curl -LsSf https://astral.sh/uv/install.sh | sh
uv venv --python 3.12
source .venv/bin/activate
uv pip install -r requirements.txt
uv pip install ipykernel
```

### 导入依赖并准备项目

本节我们将：
- 导入 `Path`、`logging` 等标准库；
- 导入 Starter Toolkit SDK：`agentkit.toolkit import sdk`；
- 准备一个示例项目目录 `tutorial_projects/t05-tool-agent`，如不存在则用 `sdk.init_project` 创建 `basic` 模板。

In [ ]:
import logging
from pathlib import Path
from agentkit.toolkit import sdk

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)
logger.info("AgentKit SDK 与 Tool SDK 已导入。")

project_root = Path("tutorial_projects/t05-tool-agent")
project_name = "tool_agent"

# print(f'约定项目路径: {project_root.resolve()}')
logger.info(f'约定项目路径: {project_root.resolve()}')

if not project_root.exists():
    # print("项目目录不存在，使用 sdk.init_project 创建示例项目...")
    logger.info("项目目录不存在，使用 sdk.init_project 创建示例项目...")
    init_result = sdk.init_project(
        project_name=project_name,
        template="basic",
        project_root=str(project_root),
    )
    logger.info("初始化是否成功:", init_result.success)
    logger.info("项目路径:", init_result.project_path)
else:
    logger.info("项目目录已存在，直接复用。")

config_file = project_root / "agentkit.yaml"
logger.info("配置文件路径: %s", config_file)


### 使用Tool SDK 创建一个沙箱工具并获取`tool_id`

In [ ]:
%env VOLCENGINE_ACCESS_KEY=
%env VOLCENGINE_SECRET_KEY=

import logging
import os

print(os.environ.get("VOLCENGINE_ACCESS_KEY"))
print(os.environ.get("VOLCENGINE_SECRET_KEY"))

from agentkit.sdk.tools.client import AgentkitToolsClient
from agentkit.sdk.tools.types import CreateToolRequest, ListToolsRequest, DeleteToolRequest

from agentkit.sdk.tools.client import (
    AgentkitToolsClient,
    CreateToolRequest,
)

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

agent_tool = AgentkitToolsClient()

create_tool_request = CreateToolRequest(
    name="tutorial-tool2",  # 替换成您的工具名称
    tool_type="All-in-one",       # 目前仅支持 "All-in-one"
    description="A simple tool to demonstrate creation.", # (可选) 添加描述
    authorizer_configuration={
        "KeyAuth": {
            "ApiKeyName": "my-api-key-20251208"
        }
    },
)

# 执行这个删掉之前存在的tool
# agent_tool.delete_tool(DeleteToolRequest(ToolId="t-yeb9ciht6oo2eybtgz0m"))

try:
    response = agent_tool.create_tool(request=create_tool_request)
    TOOL_ID = response.tool_id
    logger.info(f"已通过SDK创建工具成功! Tool ID: {TOOL_ID}")
except Exception as e:
    logger.error(f"通过SDK创建工具失败 error: {e}")

# 保存工具ID到环境变量
if TOOL_ID is not None:
    os.environ["AGENTKIT_TOOL_ID"] = TOOL_ID


### 查询Tool列表与详情

可以通过 Tool SDK 查询当前已创建的沙箱工具列表，以及某个具体工具的详情信息：
- `ListToolsRequest`：查询所有已创建的工具，支持自定义筛选条件批量查询
- `GetToolRequest`：按`tool_id`获取单个工具的详情

In [ ]:
from agentkit.sdk.tools.types import (
    FiltersItemForListTools,
    ListToolsRequest,
    GetToolRequest,
    )

# 列出部分工具
list_req = ListToolsRequest(
    filters=[
        FiltersItemForListTools(
            # name="tutorial-tool", # 精确筛选名称为"tutorial-tool"的工具
            # name和name_contains不能同时生效，两者都指定时，仅生效name
            name_contains="tutorial", # 筛选名称包含"tutorial"的工具
            values=["Ready"], # 筛选状态为"Ready"的工具
        ),
    ],
    page_number=1,
    page_size=10,
)
list_resp = agent_tool.list_tools(list_req)

tools = list_resp.tools or []
logger.info(f"共返回{len(tools)}个工具 (仅展示前10条)")
for tool in tools[:10]:
    logger.info(f"工具名称: {tool.name}, 工具ID: {tool.tool_id}, 工具状态: {tool.status}")


# 获取刚刚创建的Tool详情
get_req=GetToolRequest(
    tool_id=os.getenv("AGENTKIT_TOOL_ID",""),
)
get_resp = agent_tool.get_tool(get_req)
logger.info(f"获取当前教程使用的Tool详情")
logger.info(f"Tool ID: {get_resp.tool_id}, 工具名称: {get_resp.name}, 工具状态: {get_resp.status}")

### Agent端如何集成沙箱工具

以上步骤准备好了项目和沙箱工具，接下来我们需要在Agent端集成沙箱工具。

如果您是用 `sdk.init_project`，从 `basic` 模板创建的项目，可以参考以下示例，对入口文件（`agentkit.yaml`中的`entry_point`,通常是`<project_name>.py`）进行修改，来集成沙箱工具。

In [ ]:
# Copyright (c) 2025 Beijing Volcano Engine Technology Co., Ltd. and/or its affiliates.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import logging
import nest_asyncio
nest_asyncio.apply()

from veadk import Agent, Runner

from agentkit.apps import AgentkitSimpleApp
from veadk.prompts.agent_default_prompt import DEFAULT_DESCRIPTION, DEFAULT_INSTRUCTION
from veadk.tools.builtin_tools.run_code import run_code # (修改1) 导入运行代码工具

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)


app = AgentkitSimpleApp()

agent_name = "Agent"
description = DEFAULT_DESCRIPTION
system_prompt = DEFAULT_INSTRUCTION


# from veadk.tools.builtin_tools.web_search import web_search
# tools.append(web_search)


agent = Agent(
    name=agent_name,
    description=description,
    instruction=system_prompt,
    tools=[run_code], # (修改2) 集成沙箱工具
)
runner = Runner(agent=agent)

@app.entrypoint
async def run(payload: dict, headers: dict) -> str:
    prompt = payload["prompt"]
    user_id = headers["user_id"]
    session_id = headers["session_id"]

    logger.info(
        f"Running agent with prompt: {prompt}, user_id: {user_id}, session_id: {session_id}"
    )
    response = await runner.run(messages=prompt, user_id=user_id, session_id=session_id)

    logger.info(f"Run response: {response}")
    return response


@app.ping
def ping() -> str:
    return "pong!"


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000)


> 提示：请确保已设置必要的环境变量 `VOLC_ENGINE_ACCESS_KEY`, `VOLC_ENGINE_SECRET_KEY`, `AGENTKIT_TOOL_ID`。

## 运行代码工具

> 运行代码工具可以让智能体在运行时执行代码，例如调用 API、查询数据库等。

In [ ]:
# 原来的代码
from veadk import Agent, Runner
import asyncio
from veadk.tools.builtin_tools.run_code import run_code

agent: Agent = Agent(
    tools=[run_code],
)

runner = Runner(agent=agent)
built_in_tools_session = "run_code_tool_session"
user_prompt = "给我一个50和100之间的随机质数"
response = asyncio.run(
    runner.run(messages=user_prompt, session_id=built_in_tools_session)
)
print(response)

### 沙箱浏览器集成示例

In [ ]:
import asyncio

from veadk import Agent, Runner
from veadk.tools.builtin_tools.mcp_router import mcp_router

agent: Agent = Agent(
    tools=[mcp_router],
)

runner = Runner(agent=agent)
built_in_tools_session = "built_in_tools_session"
user_prompt = "打开火山引擎官网并查看最新的豆包模型是什么"
response = asyncio.run(
    runner.run(messages=user_prompt, session_id=built_in_tools_session)
)
print(response)

执行结果如下